# Load data
_Note_: tempo = bpm

In [34]:
import pandas as pd

KEEP_COLS_MASTER = ["name", "album", "artist", "release_date", "length",
             "tempo", "key", "time_signature",
             "mood"]

KEEP_COLS_MODEL = ["name", "release_date", "length", "tempo",
                   "key", "time_signature", "mood"]

# songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MASTER)
# make names the indices
songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)
songs_df.set_index("name", inplace=True)
songs_df

/tmp/ipykernel_332132/3311514064.py:12: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  songs_df = pd.read_csv("data_moods.csv", usecols=KEEP_COLS_MODEL, parse_dates=True, infer_datetime_format=True)


,release_date,length,tempo,key,time_signature,mood
name,,,,,,
1999,1982-10-27,379266,118.523,5,4,Happy
23,2007-04-16,318800,120.255,8,4,Sad
9 Crimes,2006-11-06,217946,136.168,0,4,Sad
99 Luftballons,1984-08-21,233000,193.100,4,4,Happy
A Boy Brushed Red Living In Black And White,2004-01-01,268000,169.881,1,4,Energetic
...,...,...,...,...,...,...
windcatcher,2020-06-19,123066,129.736,0,3,Calm
yellow is the color of her eyes,2019-11-19,435080,80.537,11,4,Sad
you broke me first,2020-04-17,169265,124.099,4,4,Sad


Convert dates to numbers, specfically seconds.

_Note_: This goddamn dataset only gives the year for some songs.

In [35]:
print(songs_df[songs_df["release_date"]=="1993"])
# credit https://stackoverflow.com/a/54312941
songs_df["release_date"] = pd.to_datetime(songs_df["release_date"], format="mixed").astype(int)/ 10**9
songs_df["release_date"]

       release_date  length    tempo  key  time_signature mood
name                                                          
Alison         1993  231893  101.571    4               4  Sad


name
1999                                           4.045248e+08
23                                             1.176682e+09
9 Crimes                                       1.162771e+09
99 Luftballons                                 4.618944e+08
A Boy Brushed Red Living In Black And White    1.072915e+09
                                                   ...     
windcatcher                                    1.592525e+09
yellow is the color of her eyes                1.574122e+09
you broke me first                             1.587082e+09
you were good to me                            1.556842e+09
æfre                                           1.594944e+09
Name: release_date, Length: 686, dtype: float64

Model will take data as (`X_train`,`y_train`)
- `X_train` is a 2D NumPy array of shape `n_samples` x `n_features`, i.e. one song per row, one metadata feature per column
- `y_train` is just convert the "mood" column into a 1D NumPy array

In [36]:
# How much should we take for training, testing?
songs_df.index.size

0.7 * songs_df.index.size

480.2

In [37]:
import numpy as np
from sklearn.model_selection import train_test_split

# use a constant random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(songs_df.drop(columns=["mood"]), songs_df[["mood"]],
                                                    test_size=0.33, random_state=42)

Make the model
Use [a i[stogram-Based Gradient Boosted Tree Classifier.
](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html#sklearn.ensemble.HistGradientBoostingClassifier.fit)

In [61]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgbt = HistGradientBoostingClassifier(max_iter=90, max_bins=30)
classification = hgbt.fit(X_train, y_train)

/home/tom/miniconda3/envs/cmpt353/lib/python3.10/site-packages/sklearn/utils/validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [62]:
classification.score(X_test, y_test) 

0.6519823788546255